In [94]:
import pandas as pd
import numpy as np
from sklearn import decomposition

udata = pd.read_table('ml-100k/u.data', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])

R = pd.DataFrame(index=range(0,943),columns=range(0,1682))
R = R.fillna(0)

for row in udata.itertuples():
    R[row[1]-1][row[2]-1] = row[3]

W = R.copy()
W[W > 0] = 1

In [97]:
### Problem 1 ###
k = [10, 50, 100]
NMF_model10 = decomposition.NMF(k[0])
NMF_model50 = decomposition.NMF(k[1])
NMF_model100 = decomposition.NMF(k[2])

U10 = NMF_model10.fit_transform(R)
V10 = NMF_model10.components_
U50 = NMF_model50.fit_transform(R)
V50 = NMF_model50.components_
U100 = NMF_model100.fit_transform(R)
V100 = NMF_model100.components_

def lse(w, r, u, v):
    error = w*((r-u.dot(v))**2)
    return error.sum().sum()
    
err10 = lse(W, R, U10, V10)
err50 = lse(W, R, U50, V50)
err100 = lse(W, R, U100, V100)

In [98]:
print "For k = %d, the least squared error is %d" % (k[0], err10)
print "For k = %d, the least squared error is %d" % (k[1], err50)
print "For k = %d, the least squared error is %d" % (k[2], err100)

For k = 10, the least squared error is 136098
For k = 50, the least squared error is 95607
For k = 100, the least squared error is 69564


In [ ]:
### Problem 2 ###
# k-fold
from scipy.sparse import csr_matrix
from sklearn.cross_validation import KFold
import matplotlib.pyplot as plt

A = csr_matrix(R.values)
N = A.getnnz()
nonzero_indices = zip(A.nonzero()[0], A.nonzero()[1])

def abs_err(actual, u, v, w, size):
    return abs(w*(u.dot(v) - actual)).sum().sum() / size

R[R==0] = np.nan
kf = KFold(n=N, n_folds=10, shuffle=True)
for train_index, test_index in kf:
#     train = R.copy()
#     train.loc[A.nonzero()[0][test_index], A.nonzero()[1][test_index]] = np.nan
#     w = pd.DataFrame(index=range(0,943),columns=range(0,1682))
#     w.fillna(0)
#     w.loc[A.nonzero()[0][test_index], A.nonzero()[1][test_index]] = 1
#     test_size = len(test_index)
#     u = NMF_model100.fit_transform(train)
#     v = NMF_model100.components_
#     print abs_err(R, u, v, w, test_size)
    
    ### Problem 3 ###
    precs = []
    recalls = []
    for threshold in range(1,6):
        print "Threshold = %d" % threshold
        test_result = u.dot(v)
        pred = test_result.lookup(A.nonzero()[0][test_index], A.nonzero()[1][test_index])
        true = R.lookup(A.nonzero()[0][test_index], A.nonzero()[1][test_index])
        precision = np.sum(true[pred > threshold] > threshold) / float(np.sum(pred > threshold)) * 100
        print "precision: %.2f\%" % precision 
        recall = np.sum(pred[true > threshold] > threshold) / float(np.sum(true > threshold)) * 100
        print "recall: %.2f\%" % recall 
        precs.append(precision)
        recalls.append(recall)
    cla()
    plt.plot(recalls, precs)
    plt.title("ROC Precision over Recall")
    plt.xlabel('Recall (\%)')
    plt.ylabel('Precision (\%)')
    

In [131]:
len(test_index)

158613